In [1]:
from vk_api import VkApi
import vk_api
import vk
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
# from nltk.tokenize.casual import TweetTokenizer
# from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
import pymorphy2
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from keras.models import Sequential
from keras.layers import Dense, Dropout
from nltk.corpus import stopwords

# Прогноз количества комментариев постов 

In [2]:
# id групп вк

id_avia = '-39283725'
id_moscow = '-137324015'
id_nazem = '-212086355'
mas_id = [id_avia, id_moscow, id_nazem]
token = 'vk1.a.J6_ahUxPyWhFaYbShB5hkJ4hAYpwRE8oanqh1vLBPZOQ8bo7RzICWE1yuEhUs1iMWYTI-zalDHRr8V6NAS-12S-qYImDQT7sVejSI3_3SH1uaAIWrxkC5tljaQDAwd7O17rCrccRDEvf09rreDLBfPsVhRCQYdK_Yelswjs1homcD1B6MFHTVqvrfmyV2gZ_30kGH19ZsdDn3IA0EWZszg'


Парсинг данных из групп вк

In [3]:

# with open('C:/Users/Admin/Desktop/token.txt') as f:
#      token = f.readlines()[0]

#token = 'https: // oauth.vk.com/blank.html # access_token=vk1.a.J6_ahUxPyWhFaYbShB5hkJ4hAYpwRE8oanqh1vLBPZOQ8bo7RzICWE1yuEhUs1iMWYTI-zalDHRr8V6NAS-12S-qYImDQT7sVejSI3_3SH1uaAIWrxkC5tljaQDAwd7O17rCrccRDEvf09rreDLBfPsVhRCQYdK_Yelswjs1homcD1B6MFHTVqvrfmyV2gZ_30kGH19ZsdDn3IA0EWZszg&expires_in=0&user_id=257606113'
token = token


def main(offset: int, token: str, group_id: str):
    vk = vk_api.VkApi(token=token)  # авторизация через токен (желательно)
    api = vk.get_api()
    posts = api.wall.get(owner_id=group_id, offset=offset, count=100)['items']
    posts_strings = [post['text'] for post in posts]
    num_like = []
    comments_strings = []
    for post in posts:
        comments = api.wall.getComments(
            owner_id=group_id, post_id=post['id'], count=100)['items']
        comments_strings.append([comment['text'] for comment in comments])
        itemID = post['id']
        isLiked = api.likes.getList(
            type='post',
            owner_id=group_id,
            item_id=itemID
        )
        num_like.append(isLiked['count'])
    return posts_strings, comments_strings, num_like


combo_list_posts = []
combolist_comments = []
combolist_like = []

for group_id in mas_id:
    for i in range(0, 301, 100):
        try:
            rzd_posts, comments_strings_rzd, rzd_like_count = main(
                offset=i, token=token, group_id=group_id)
            combo_list_posts.extend(rzd_posts)
            combolist_comments.extend(comments_strings_rzd)
            combolist_like.extend(rzd_like_count)
            print(group_id, i)
        except:
            print('Постов больше нет на смещении: ', i)

       


-39283725 0
-39283725 100
-39283725 200
Постов больше нет на смещении:  300
Постов больше нет на смещении:  0
-137324015 100
-137324015 200
-137324015 300
-212086355 0
-212086355 100
-212086355 200
-212086355 300


In [4]:
combolist_comments_count = [len(comments) for comments in combolist_comments]


In [5]:
df = pd.DataFrame({'Посты': combo_list_posts,
                  'Комментарии': combolist_comments_count})
df


,Посты,Комментарии
0,Петербургский Пулково (LED) стал двадцатым аэр...,1
1,Выбравшая путь полной импортонезависимости в с...,1
2,"Компания ""Сирена"", ведущий российский поставщи...",1
3,Туркменская авиакомпания Turkmenistan Airlines...,1
4,"Кэмпбелл Уилсон, президент Air India, в очеред...",1
...,...,...
995,,1
996,🖥В парках Мосгортранса начали пользоваться ПАК...,1
997,🚐Мосгортранс протестирует электробус малого кл...,19
998,,4


In [8]:
df = df.drop_duplicates()
df.isnull().sum()



Посты          0
Комментарии    0
dtype: int64

In [9]:
df['Комментарии'].mean()


5.034979423868313

In [10]:
df = df.astype({'Посты': np.str})


C:\Users\Admin\AppData\Local\Temp\ipykernel_6260\2142041351.py:1: DeprecationWarning: `np.str` is a deprecated alias for the builtin `str`. To silence this warning, use `str` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.str_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  df = df.astype({'Посты': np.str})


Токенизация

In [11]:

nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')

stop_words = set(stopwords.words('russian'))
lemmatizer = WordNetLemmatizer()
morph = pymorphy2.MorphAnalyzer()


def preprocess_text(text):
    # Токенизация
    tokens = word_tokenize(text)

    # Приведение к нижнему регистру
    tokens = [token.lower() for token in tokens]

    # Лемматизация
    tokens = [lemmatizer.lemmatize(token) for token in tokens]

    # Приведение к начальной форме
    tokens = [morph.parse(token)[0].normal_form for token in tokens]

    # Удаление стоп-слов
    tokens = [token for token in tokens if token not in stop_words]

    # Объединение токенов в строку
    clean_text = ' '.join(tokens)

    return clean_text


df['clean_text'] = df['Посты'].apply(preprocess_text)


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [12]:
df

,Посты,Комментарии,clean_text
0,Петербургский Пулково (LED) стал двадцатым аэр...,1,петербургский пулково ( led ) стать двадцатый ...
1,Выбравшая путь полной импортонезависимости в с...,1,выбрать путь полный импортонезависимость страт...
2,"Компания ""Сирена"", ведущий российский поставщи...",1,"компания `` сирена '' , ведущий российский пос..."
3,Туркменская авиакомпания Turkmenistan Airlines...,1,туркменский авиакомпания turkmenistan airline ...
4,"Кэмпбелл Уилсон, президент Air India, в очеред...",1,"кэмпбелла уилсон , президент air india , очере..."
...,...,...,...
993,🚋Один «Витязь» весит как 40 легковушек или ста...,7,🚋один « витязь » весить 40 легковушка стадо по...
994,🐱Сотрудники «Организатора перевозок» взяли под...,1,🐱сотрудник « организатор перевозка » взять опе...
996,🖥В парках Мосгортранса начали пользоваться ПАК...,1,🖥в парка мосгортранс начать пользоваться пак —...
997,🚐Мосгортранс протестирует электробус малого кл...,19,🚐мосгортранс протестировать электробус малое к...


In [22]:

text_data = df['clean_text']
labels = df['Комментарии']

# Разделение данных на тренировочный и тестовый наборы
X_train, X_test, y_train, y_test = train_test_split(
    text_data, labels, test_size=0.2, random_state=42)

# Векторизация текстовых данных
vectorizer = TfidfVectorizer()
X_train = vectorizer.fit_transform(X_train).toarray() 
X_test = vectorizer.transform(X_test).toarray()

# Создание модели нейронной сети
model = Sequential()
model.add(Dense(100, activation='relu', input_dim=X_train.shape[1]))
model.add(Dropout(0.2))
model.add(Dense(64, activation='relu'))
model.add(Dense(1, activation='linear'))

# Компиляция модели
model.compile(loss='mean_squared_error', optimizer='adam', metrics=['mae'])

# Обучение модели
model.fit(X_train, y_train, epochs=10, batch_size=64)



Epoch 1/10
13/13 [==============================] - 2s 22ms/step - loss: 65.7057 - mae: 4.8839
Epoch 2/10
13/13 [==============================] - 0s 19ms/step - loss: 60.6451 - mae: 4.5000
Epoch 3/10
13/13 [==============================] - 0s 21ms/step - loss: 51.6212 - mae: 3.9422
Epoch 4/10
13/13 [==============================] - 0s 20ms/step - loss: 41.9745 - mae: 3.5829
Epoch 5/10
13/13 [==============================] - 0s 19ms/step - loss: 35.1649 - mae: 3.4509
Epoch 6/10
13/13 [==============================] - 0s 18ms/step - loss: 30.5630 - mae: 3.0758
Epoch 7/10
13/13 [==============================] - 0s 20ms/step - loss: 26.1853 - mae: 2.5253
Epoch 8/10
13/13 [==============================] - 0s 23ms/step - loss: 22.3461 - mae: 2.2298
Epoch 9/10
13/13 [==============================] - 0s 20ms/step - loss: 18.9460 - mae: 1.8542
Epoch 10/10
13/13 [==============================] - 0s 20ms/step - loss: 16.0621 - mae: 1.5870


In [21]:
loss, mae = model.evaluate(X_test, y_test)
print(f'Test Loss: {loss}, Test MAE: {mae}')


7/7 [==============================] - 0s 2ms/step - loss: 50.0647 - mae: 3.1415
Test Loss: 50.064735412597656, Test MAE: 3.1414921283721924
